# Neural Network Adapters for faster low-memory fine-tuning

This tutorial covers the SpeechBrain implementation of adapters such as LoRA. This includes how to integrate either SpeechBrain implemented adapters, custom adapters, and adapters from libraries such as PEFT into a pre-trained model.

## Prerequisite
- [Speech Recognition From Scratch](https://speechbrain.readthedocs.io/en/latest/tutorials/tasks/speech-recognition-from-scratch.html)

## Introduction and Background

As pre-trained models become larger and more capable, there is growing interest in methods for adapting them for specific tasks in a memory-efficient way, within a reasonable time span. One such technique is freezing the original parameters and inserting a small number of additional parameters into the original model, which are called "adapters." These adapters can often match the performance of full fine-tuning at a fraction of the parameter count, meaning faster and more memory-efficient fine-tuning [1]. One popular technique for doing this is known as Low-Rank Adaptation (LoRA) [2].

On the software side, HuggingFace has produced a popular library for adapters called PEFT [3]. Our implementation includes some of the features of this library, as well as including the ability to integrate PEFT adapters into a SpeechBrain model. To explore this further, let's proceed with the installation of SpeechBrain.

### Relevant bibliography
1. N. Houlsby, A. Giurgiu, S. Jastrzebski, B. Morrone, Q. De Laroussilhe, A. Gesmundo, M. Attariyan, and S. Gelly, "Parameter-efficient transfer learning for NLP." In *International Conference on Machine Learning*, 2019.
2. E.J. Hu, P. Wallis, Z. Allen-Zhu, Y. Li, S. Wang, L. Wang, and W. Chen, "LoRA: Low-rank adaptation of large language models." In *International Conference on Learning Representations*, 2021.
3. S. Mangrulkar, S. Gugger, L. Debut, Y. Belkada, S. Paul, and  B. Bossan, "PEFT: State-of-the-art parameter-efficient fine-tuning methods." *GitHub Repository*, 2022.


In [3]:
%%capture
#!python -m pip install git+https://github.com/speechbrain/speechbrain.git@develop

!git clone https://github.com/speechbrain/speechbrain.git

## Simple Fine-tuning

We'll first show how to use adapters on a template recipe, which includes everything necessary for full training.

In [14]:
#%cd speechbrain/templates/speech_recognition/ASR
!python train.py train.yaml --number_of_epochs=1  --batch_size=2  --enable_add_reverb=False --enable_add_noise=False #To speed up

INFO:speechbrain.utils.seed:Setting seed to 2602
/home/competerscience/Documents/Repositories/speechbrain/speechbrain/utils/autocast.py:68: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)
speechbrain.core - Beginning experiment!
speechbrain.core - Experiment folder: results/CRDNN_BPE_960h_LM/2602
mini_librispeech_prepare - Creating ../train.json, ../valid.json, and ../test.json
mini_librispeech_prepare - Transcription files read!
../data/LibriSpeech/train-clean-5/669/129074/669-129074-0015.flac
True
/home/competerscience/Documents/Repositories/speechbrain/docs/tutorials/nn/speechbrain/templates/speech_recognition/ASR
speechbrain.core - Exception:
Traceback (most recent call last):
  File "/home/competerscience/Documents/Repositories/speechbrain/docs/tutorials/nn/speechbrain/templates/speech_recognition/ASR/train.py", line 454

In [11]:
!ls ../data/LibriSpeech/train-clean-5/669/129074/669-129074-0015.flac

../data/LibriSpeech/train-clean-5/669/129074/669-129074-0015.flac


In [13]:
import os
os.getcwd()

'/home/competerscience/Documents/Repositories/speechbrain/docs/tutorials/nn/speechbrain/templates/speech_recognition/ASR'

In [15]:
import torchaudio

In [17]:
!uv install ffmpeg
torchaudio.load("../data/LibriSpeech/train-clean-5/669/129074/669-129074-0015.flac")

error: unrecognized subcommand 'install'

  tip: a similar subcommand exists: 'uv pip install'

Usage: uv [OPTIONS] <COMMAND>

For more information, try '--help'.


RuntimeError: Error loading audio file: failed to open file ../data/LibriSpeech/train-clean-5/669/129074/669-129074-0015.flac